In [118]:
import pandas as pd
import numpy as np

EndDate = '2021-12-25'


######## input your forecast/budget number to the list below ###########
     
            # Desc         Amt  Start Date  Freq 
mylist = [['The Pond Rent   ',650,'2021-01-01','7D'],
          ['Wolli Creek Rent',450,'2021-01-01','7D'],
          ['Salary          ',450,'2021-01-01','7D'],
         ]

########################################################################


df = pd.DataFrame(columns=['Date','Description','Amount'])

for i in range(len(mylist)):
    date = pd.date_range(mylist[i][2],EndDate,freq=mylist[i][3])
    amount = mylist[i][1]
    desc = mylist[i][0]
    tmp = pd.DataFrame({'Date':date,'Description':desc,'Amount':amount})
    df = df.append(tmp)   
    
# add Period
df['Period'] = pd.to_datetime(df['Date']).dt.to_period('M')

Total = df[['Period','Description','Amount']]
Total.groupby(['Period','Description']).sum().reset_index()


,Period,Description,Amount
0,2021-01,Salary,2250
1,2021-01,The Pond Rent,3250
2,2021-01,Wolli Creek Rent,2250
3,2021-02,Salary,1800
4,2021-02,The Pond Rent,2600
5,2021-02,Wolli Creek Rent,1800
6,2021-03,Salary,1800
7,2021-03,The Pond Rent,2600
8,2021-03,Wolli Creek Rent,1800
9,2021-04,Salary,2250


In [194]:

def cat(string):
    
# list of keywords, add as you find new ones
    WolliRent = ['slick','carena']
    ThePondRent    = ['Harcourts','Hills Prestige','Hillside']
    Grocery          = ['Coles','Woolworths','Woolworth']
    Shopping         = ['REBEL','Chemist']
    Salary           = ['Salary']
    Internet         = ['TPG']

    if string in ' '.join(Grocery):
        cat = 'Grocery'
    elif string in ' '.join(Salary):
        cat = 'Salary'
    elif string in ' '.join(Shopping):
        cat = 'Internet'
    else:
        cat = 'Other'

    return cat

In [ ]:
x for x in xyz if x not in a

In [236]:
a = [2,3,4,5,6,7,8,9,0]
xyz = [0,12,4,6,242,7,9]

print([x for x in xyz if x in a])

[0, 4, 6, 7, 9]


In [255]:

grocery = ['coles','cole','tongli']
electronic = ['jb','harvery']

mytext = ['this is cole']

for i in grocery:
    if i in ''.join(mytext):
        print (i)
        break

cole


In [257]:
xyz = ['coles','cole','tongli']
a = ['this is coles']

check = [x for x in xyz if x in ''.join(a)]

for i in check:
    cat = 'Groceries'
    print(cat)

SyntaxError: invalid syntax (<ipython-input-257-6e2e28811905>, line 4)

In [235]:
s for s in some_list if myword in s

SyntaxError: invalid syntax (<ipython-input-235-bd6988c81078>, line 1)

In [230]:
some_list  = ['Coles','Cole']
#s = 'Rebel BONDI BONDI JUNCTINSW'
if any("Rebel" in s for s in some_list):
    print ('OK')
        

In [ ]:
j = input('what:')

print (j)

In [ ]:
cba = pd.read_csv('/users/ronny/downloads/CSVData-8.csv',header=None,usecols=[0,1,2])
cba.columns=['Date','Amount','Desc']

In [196]:
cba['Cat'] = cba.apply(lambda x:cat(x['Desc']),axis=1)

cba

,Date,Amount,Desc,Cat
0,14/01/2021,-54.80,L J HARPER & CO PTY KIRRAWEE,Other
1,13/01/2021,-8.20,WW METRO 1716 HAYMARKET NS,Other
2,13/01/2021,-10.55,THAI KEE IGA HAYMARKET NSW,Other
3,13/01/2021,-5.99,TONGLI SUPERMARKET QPS HAYMARKET,Other
4,12/01/2021,-6.30,ANDERSEN'S OF DENMAR COOGEE NSW,Other
...,...,...,...,...
115,17/12/2020,-69.99,REBEL BONDI BONDI JUNCTINSW,Other
116,17/12/2020,-4.98,CHEMIST WAREHOUSE HAYMARKET NSW,Other
117,17/12/2020,-21.25,WW METRO 1716 HAYMARKET NS,Other
118,17/12/2020,-18.00,KMART 1084 BONDI JCT AUS,Other


In [158]:
l = 'rebel is the sport'

l.find('is')

6

In [162]:
'foo' in '**foo**'

True

In [160]:
'foo' in '**foo**'

True

In [180]:
s = ' '.join(grocery)
'Cole' in s

True

In [179]:
s

'Coles Woolworths'

In [167]:
'cole' in 'colesWoolwortjs'

True